# The Experiment Container

This notebook explains how the database works as an experiment container.

In [1]:
import qcodes as qc

## The database

Already at import time, the database on file is selected via the QCoDeS configuration file. There is thus always only one database selected as the "active" one.


In [2]:
# To inspect it, use
configuration = qc.config
print(configuration['core']['db_location'])

./experiments.db


Note that the default setting for the database location is a relative path. That is a sane default setting, but not always what you want. If you execute scripts from many different folders on your machine, you should change the default location to an absolute path pointing to your data folder, or you will end up with multiple databases.

In [3]:
# Changing the settings can done like so:

configuration['core']['db_location'] = '~/my/particular/absolute/path/database.db'
configuration.save_to_home()  # this saves to the configuration file on disk

You can try restarting the kernel and verify that the changes took effect. For the sake of pedagogical clarity, we now create a new empty default database.

In [4]:
configuration['core']['db_location'] = './exp_container_tutorial.db'
configuration.save_to_home()
# ...restart the kernel... and verify:

      **kernel was restarted**

In [1]:
import qcodes as qc
configuration = qc.config
print(configuration['core']['db_location'])

./exp_container_tutorial.db


## The experiments inside the database

The database holds a certain number of **experiments**. They may be viewed:

In [2]:
qc.dataset.experiment_container.experiments()

[]

Not surprisingly, our new database is empty. Let us add some experiments.

In [3]:
qc.dataset.experiment_container.new_experiment('first_exp', sample_name='old_sample')
qc.dataset.experiment_container.new_experiment('second_exp', sample_name='slightly_newer_sample')
qc.dataset.experiment_container.new_experiment('second_exp', sample_name='brand_new_sample')

second_exp#brand_new_sample#3@./exp_container_tutorial.db
---------------------------------------------------------

In [4]:
qc.dataset.experiment_container.experiments()

[first_exp#old_sample#1@./exp_container_tutorial.db
 --------------------------------------------------,
 second_exp#slightly_newer_sample#2@./exp_container_tutorial.db
 --------------------------------------------------------------,
 second_exp#brand_new_sample#3@./exp_container_tutorial.db
 ---------------------------------------------------------]

We notice that each experiment is labelled by an integer. This is the **exp_id** that can be used when looking up properties of each experiment.

Let us add some runs to experiment 2 ("second_exp"). In this tutorial, we don't bother to actually add data to the runs.

In [5]:
qc.dataset.data_set.new_data_set('a_good_run', exp_id=2)

a_good_run #1@./exp_container_tutorial.db
-----------------------------------------

In [6]:
qc.dataset.data_set.new_data_set('a_nother_run', exp_id=2)

a_nother_run #2@./exp_container_tutorial.db
-------------------------------------------

We may now inspect experiment 2.

In [7]:
exp_2 = qc.dataset.experiment_container.load_experiment(2)

In [8]:
# Printing the experiment will reveal the runs

print(exp_2)

second_exp#slightly_newer_sample#2@./exp_container_tutorial.db
--------------------------------------------------------------
1-a_good_run-1-None-0
2-a_nother_run-2-None-0


In a similar way, we may of course add runs to the other two experiments. Now, the alert reader will have noticed that `exp_id` is a keyword argument in `new_data_set`. What is the default then? --The default is that the run is added to the experiment with the **highest** `exp_id` in the database.